In [1]:
import pymorphy2
import json
from collections import defaultdict, Counter

In [2]:
morph = pymorphy2.MorphAnalyzer()

переенёс в скрипт Normalize_drugnames_and_diseasenames.py

# выбираем не начальную форму, а самый популярный вариант этой начальной формы

In [95]:
norm_and_variations_drug = defaultdict(Counter)
norm_and_variations_disease = defaultdict(Counter)
with open("../data/raw/ner_800k_jsonl/short_ner_800k_norms.jsonl", "r") as outf:
    for line in outf:
        docData = json.loads(line)
        for k, v in docData["entities"].items():
            if "Drugname" in v["tag"]:
                norm_and_variations_drug[v["norm_form"]][v["text"].lower()]+=1
            elif "Diseasename" in v["tag"]:
                norm_and_variations_disease[v["norm_form"]][v["text"].lower()]+=1

In [96]:
correct_pairs_drug = {}
for k, v in norm_and_variations_drug.items():
    correct_pairs_drug[k] = v.most_common(1)[0][0]
correct_pairs_dis = {}
for k, v in norm_and_variations_disease.items():
    correct_pairs_dis[k] = v.most_common(1)[0][0]

In [97]:
lines = []
with open("../data/raw/ner_800k_jsonl/short_ner_800k_norms.jsonl", "r") as inf, open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_2.jsonl", "w") as outf:
    counter = 0
    for line in inf:
        if counter % 1000 == 0:
            pass
            #print(counter)
        counter += 1
        docData = json.loads(line)
        for k, v in docData["entities"].items():
            if "Drugname" in v["tag"]:
                v["norm_form"] = correct_pairs_drug[v["norm_form"]]
            elif "Diseasename" in v["tag"]:
                v["norm_form"] = correct_pairs_dis[v["norm_form"]]
        outf.write(json.dumps(docData) + "\n")

# add meddra from gleb

In [3]:
with open("/s/ls4/users/romanrybka/pharm_er/Pipeline_Ner_Norm/RelationExtraction/ner_norm_800k.jsonl", "r") as f:
    corpus = json.load(f)

In [5]:
corpus[1]

{'text_id': 1,
 'entities': {'0': {'text': 'подействовал после первого же применения',
   'tag': 'Disease:DisTypeBNE-Pos'},
  '1': {'text': 'перестала кашлять раз', 'tag': 'Disease:DisTypeBNE-Pos'},
  '2': {'text': 'простуды', 'tag': 'Disease:DisTypeDiseasename'},
  '3': {'text': 'кашляла',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA': 0,
   'MedDRA_code': ['Кашель', '10011224']},
  '4': {'text': 'больного горла',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA': 1,
   'MedDRA_code': ['Боль в ротоглотке (орофарингеальная)', '10068319']},
  '5': {'text': 'кашлять',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA': 2,
   'MedDRA_code': ['Кашель', '10011224']},
  '6': {'text': 'кашлем',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA': 3,
   'MedDRA_code': ['Кашель', '10011224']},
  '7': {'text': 'кашель приступообразный',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA': 4,
   'MedDRA_code': ['Кашель', '10011224']},
  '8': {'text': 'кашле',
   'tag': 'Disease:DisTypeIndicat

In [100]:
meddra_synonyms = defaultdict(set)
with open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_2.jsonl", "r") as origf, \
    open("/s/ls4/users/romanrybka/pharm_er/Pipeline_Ner_Norm/RelationExtraction/ner_norm_800k.jsonl", "r") as normf, \
    open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_3.jsonl", "w") as outf:
    counter = 0
    corpus = json.load(normf)
    for line in origf:
        docData = json.loads(line)
        assert docData["text_id"]==corpus[counter]["text_id"]
        for k in docData["entities"].keys():
            assert docData["entities"][k]["text"]==corpus[counter]["entities"][k]["text"]
            if "MedDRA_code" in corpus[counter]["entities"][k]:
                assert len(corpus[counter]["entities"][k]["MedDRA_code"])==2
                docData["entities"][k]["MedDRA_code"] = corpus[counter]["entities"][k]["MedDRA_code"]
                meddra_synonyms[corpus[counter]["entities"][k]["MedDRA_code"][0]].add(docData["entities"][k]["text"].lower())
        outf.write(json.dumps(docData)+"\n")
        counter += 1

# добавить url из файла

In [113]:
with open("../data/raw/ner_800k_jsonl/urls.json", "r") as f:
    urld_list = json.load(f)
with open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_3.jsonl", "r") as origf, \
     open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_4.jsonl", "w") as outf:
    i = 0
    for line in origf:
        docData = json.loads(line) 
        docData["meta"] = {}
        docData["meta"]["url"] = urld_list[i]
        i+=1
        
        outf.write(json.dumps(docData) + "\n")

In [114]:
urld_list[-1]

'https://otzovik.com/review_4729315.html'

In [116]:
docData

{'text_id': 770110,
 'entities': {'0': {'text': 'результатом уже довольна',
   'tag': 'Disease:DisTypeBNE-Pos'},
  '1': {'text': 'кожа уже гораздо лучше конец дня мой нос блестит ярче солнца',
   'tag': 'Disease:DisTypeBNE-Pos'},
  '2': {'text': 'воспаления на лице',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Воспаление', '10061218']},
  '3': {'text': 'Avene Cleanance', 'tag': 'Medication:MedFromForeign'},
  '4': {'text': 'Фармленде', 'tag': 'Medication:MedMakerDomestic'},
  '5': {'text': 'эмульсию', 'tag': 'Medication:MedTypeDrugform'},
  '6': {'text': 'гелю', 'tag': 'Medication:MedTypeDrugform'},
  '7': {'text': 'лосьону', 'tag': 'Medication:MedTypeDrugform'},
  '8': {'text': 'гель', 'tag': 'Medication:MedTypeDrugform'},
  '9': {'text': 'Avene Cleanance',
   'tag': 'Medication:MedTypeDrugname',
   'norm_form': 'avene cleanance'},
  '10': {'text': 'Фармленде', 'tag': 'Medication:MedTypeMedMaker'},
  '11': {'text': 'для умывания', 'tag': 'Medication:MedTypeRoute'},
  '1

# проверка проблемных отзывов

In [104]:
with open("../data/raw/ner_800k_jsonl/short_ner_800k_norms_3.jsonl", "r") as outf:
    for line in outf:
        docData = json.loads(line)
        if docData["text_id"]!=410728:  # 19511
            continue
        break

In [67]:
import pymorphy2
#'мукалтином' 'мукалтин'
#'мукалтину' 'мукалтина'
#'Мукалтин' 'мукалтины'
#'мукалтин' 'мукалтины'

morph = pymorphy2.MorphAnalyzer()
parse = morph.parse('мукалтин')
print("pymorphy", parse)

[Parse(word='мукалтин', tag=OpencorporaTag('NOUN,inan,femn plur,gent'), normal_form='мукалтина', score=0.2727272727272727, methods_stack=((DictionaryAnalyzer(), 'тин', 55, 8), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'мукал'))),
 Parse(word='мукалтин', tag=OpencorporaTag('NOUN,anim,femn,Name plur,gent'), normal_form='мукалтина', score=0.2727272727272727, methods_stack=((DictionaryAnalyzer(), 'тин', 69, 8), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'мукал'))),
 Parse(word='мукалтин', tag=OpencorporaTag('NOUN,anim,femn,Name plur,accs'), normal_form='мукалтина', score=0.2727272727272727, methods_stack=((DictionaryAnalyzer(), 'тин', 69, 10), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'мукал'))),
 Parse(word='мукалтин', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='мукалтин', score=0.0909090909090909, methods_stack=((FakeDictionary(), 'мукалтин', 34, 0), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'алтин'))),
 Parse(word='мукалтин', tag=Opencorp

In [107]:
from pymystem3 import Mystem
m = Mystem()
"".join(m.lemmatize('простудными заболеваниями')).strip()

'простудный заболевание'

In [106]:
import spacy
nlp = spacy.load("ru_core_news_sm")
doc = nlp('простудными заболеваниями')
for token in doc:
    print(token.lemma_)

простудный
заболевание


In [105]:
docData

{'text_id': 410728,
 'entities': {'0': {'text': 'аллергические возможные реакции',
   'tag': 'ADR',
   'MedDRA_code': ['Гиперчувствительность', '10020751']},
  '1': {'text': 'помогает кашель', 'tag': 'Disease:DisTypeBNE-Pos'},
  '2': {'text': 'действенно', 'tag': 'Disease:DisTypeBNE-Pos'},
  '3': {'text': 'простудными заболеваниями',
   'tag': 'Disease:DisTypeDiseasename',
   'norm_form': 'простудных заболеваний'},
  '4': {'text': 'кашля',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Кашель', '10011224']},
  '5': {'text': 'кашля',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Кашель', '10011224']},
  '6': {'text': 'мокрый кашель',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Кашель', '10011224']},
  '7': {'text': 'откашливать',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Кашель', '10011224']},
  '8': {'text': 'кашля',
   'tag': 'Disease:DisTypeIndication',
   'MedDRA_code': ['Кашель', '10011224']},
  '9': {'text': 'Алтейка', 'tag'